In [ ]:
%pip install PyCryptodome

In [ ]:
from Crypto.Util.number import inverse, long_to_bytes, bytes_to_long
from Crypto.Cipher import AES
from hashlib import sha256
import math

In [ ]:
p, q = (10699940648196411028170713430726559470427113689721202803392638457920771439452897032229838317321639599506283870585924807089941510579727013041135771337631951, 11956676387836512151480744979869173960415735990945471431153245263360714040288733895951317727355037104240049869019766679351362643879028085294045007143623763) 
vka = 124641741967121300068241280971408306625050636261192655845274494695382484894973990899018981438824398885984003880665335336872849819983045790478166909381968949910717906136475842568208640203811766079825364974168541198988879036997489130022151352858776555178444457677074095521488219905950926757695656018450299948207 
vkakb = 114778245184091677576134046724609868204771151111446457870524843414356897479473739627212552495413311985409829523700919603502616667323311977056345059189257932050632105761365449853358722065048852091755612586569454771946427631498462394616623706064561443106503673008210435922340001958432623802886222040403262923652 
vkb = 6568897840127713147382345832798645667110237168011335640630440006583923102503659273104899584827637961921428677335180620421654712000512310008036693022785945317428066257236409339677041133038317088022368203160674699948914222030034711433252914821805540365972835274052062305301998463475108156010447054013166491083 
c = 'fef29e5ff72f28160027959474fc462e2a9e0b2d84b1508f7bd0e270bc98fac942e1402aa12db6e6a36fb380e7b53323'

n = p * q

The key exchange for $v$ in nowayback.py is the following:
$$\begin{aligned}
v &= p * rand(1,n) &\pmod n \\
v_{k_a} &= v * k_a &\pmod n \\
v_{k_ak_b} &= v_{k_a} * k_b = v * k_a * k_b &\pmod n \\
v_{k_b} &= v_{k_ak_b} * k_a^{-1} = v * k_a * k_b * k_a^{-1} = v * k_b &\pmod n \\
v_s &= v_{k_b} * k_b^{-1} = v * k_b * k_b^{-1} = v &\pmod n
\end{aligned}$$

We can look at the same equations modulo $q$ and they will look in the same way because $q$ is a factor of $n$. This will help us because although $v$ dosen't have an inverse modulo $n$ (as $p$ is one of it's factors), it has an inverse modulo $q$.

This means we can calculate:
$$\begin{aligned}
k_a &= v_{k_a}^{-1} * v_{k_ak_b} &\pmod q \\
k_b &= v_{k_b}^{-1} * v_{k_ak_b} &\pmod q
\end{aligned}$$

In [ ]:
ka_q = (inverse(vkb, q) * vkakb) % q
kb_q = (inverse(vka, q) * vkakb) % q
ka_q, kb_q

And also to calculate $v$:
$$v = v_{k_a} * k_a^{-1} \pmod q$$

In [ ]:
v_q = (vka * inverse(ka_q,q)) % q
v_q

Let's call this value $v_q$. This means we know The following equations:
$$\begin{aligned}
v &= v_q &\pmod q \\
v &= p*rand(1,n) = 0 &\pmod p \\
\end{aligned}$$
using the Chinese Remainder Theorem we can calculate $v$.

In [ ]:
#see https://rosettacode.org/wiki/Chinese_remainder_theorem#Python
from functools import reduce
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * inverse(p, n_i) * p
    return sum % prod

In [ ]:
v = chinese_remainder([p, q], [0, v_q])
v

In [ ]:
#reversig the encryption
key = sha256(long_to_bytes(v)).digest()
cipher = AES.new(key, AES.MODE_ECB)
m = cipher.decrypt(bytes.fromhex(c))
m